In [ ]:
!pip install schnetpack

In [ ]:
!pip uninstall --y numpy
!pip install numpy==1.23

# Import Libraries

In [ ]:
import os
import time
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import numpy as np
from tqdm import tqdm

from NeuralModel import AtomNeuralNet
from NeuralModel import TrainModel
from NeuralModel import Inference

Instructions for updating:
non-resource variables are not supported in the long term


# Auxliary Functions

In [ ]:
def Euclidian_Distance(vec_a, vec_b):
  aux = vec_a - vec_b
  aux = aux**2
  return sum(aux)**0.5


def FeatureMatrix(simbols, Max_dim):
  elementos_dict =  {6: 0, 9: 1, 1: 2, 7: 3, 8: 4} #QM9
  matrix_features = np.zeros(shape=(Max_dim, len(elementos_dict)), dtype=float, order='C')

  dimensao = len(simbols)
  for i in range(dimensao):
    r = elementos_dict[simbols[i]]
    matrix_features[i,r] = 1

  return matrix_features

def Distance_Matrix(coordinates, max_dim):
  matrix_dist = np.zeros(shape=(max_dim, max_dim), dtype=float, order='C')

  n_atoms = coordinates.shape[0]

  for i in range(n_atoms):
    for j in range(i, n_atoms):
      vec_a = coordinates[i]
      vec_b = coordinates[j]
      dist = Euclidian_Distance(vec_a, vec_b)
      matrix_dist[i,j] = dist
      matrix_dist[j,i] = dist

  return matrix_dist

###  Choose the Property



In [ ]:
#molecule_property = 'U0'
#molecule_property = 'U'
molecule_property = 'H'
#molecule_property = 'G'

# Load Data
* Here we use the data available in schnetpack.

In [ ]:
if molecule_property == 'U':
  aux_prop = QM9.U
elif molecule_property == 'U0':
  aux_prop = QM9.U0
elif molecule_property == 'H':
  aux_prop = QM9.H
elif molecule_property == 'G':
  aux_prop = QM9.G

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

qm9data = QM9(
    './qm9.db',
    batch_size=32,
    num_train=110000,
    num_val=10000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        #trn.RemoveOffsets(aux_prop, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    property_units={aux_prop: 'eV'},
    num_workers=1,
    split_file=os.path.join(qm9tut, "split.npz"),
    pin_memory=True, # set to false, when not using a GPU
    load_properties=[aux_prop], #only load U0 property
)
qm9data.prepare_data()
qm9data.setup()

# Exploring the dataset
* Checking how the data is available in the dataset

In [ ]:
index_molecule = 20

In [ ]:
# Atoms positions (x,y,z) coordinates
qm9data.train_dataset[index_molecule]['_positions'].tolist()

[[1.590023398399353, 1.229581356048584, -0.5534893870353699],
 [0.561308741569519, 0.351553738117218, -0.07693459093570709],
 [0.08277510851621628, -0.5378395915031433, -1.1409991979599],
 [-0.2798251509666443, -1.2579381465911865, -2.0325896739959717],
 [1.0124551057815552, -0.3829304277896881, 1.1611121892929077],
 [1.0877143144607544, -1.7005839347839355, 1.3548387289047241],
 [1.5660992860794067, -2.047248601913452, 2.743448495864868],
 [2.1012208461761475, -0.6930356621742249, 3.279163122177124],
 [1.4320026636123657, 0.3896622359752655, 2.396336555480957],
 [2.3231191635131836, 0.6699607372283936, -0.8355581164360046],
 [-0.2535625398159027, 1.0359872579574585, 0.19725042581558228],
 [-0.6123668551445007, -1.8855717182159424, -2.8222787380218506],
 [0.7973502278327942, -2.4460525512695312, 0.6237061023712158],
 [0.7327741980552673, -2.429065704345703, 3.350898265838623],
 [2.3295035362243652, -2.833505868911743, 2.7483668327331543],
 [1.9071537256240845, -0.5511515140533447, 4.34

In [ ]:
# atomic numbers
qm9data.train_dataset[index_molecule]['_atomic_numbers'].tolist()

[8, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
# Target value
print("Target", aux_prop, qm9data.train_dataset[index_molecule][aux_prop].tolist()[0])

Target enthalpy_H -10499.634765625


In [ ]:
# Reference Values
# Reference values are constant associated to each chemical element which the model does not need to learn.
# This is why these values are removed during the data preparation.
atomic_number = 6
atomrefs = qm9data.train_dataset.atomrefs
print("Atomic number:", atomic_number, "    Reference Value:", atomrefs[aux_prop][atomic_number].item())

Atomic number: 6     Reference Value: -1029.798828125


# Prepare Data for Model Training
* Create the features matrices - one hot encoding based on chemical element
* Create the atoms distance matrix

In [ ]:
lst_features_treino = list()
lst_distancias_treino = list()
lst_mol_sizes_treino = list()
lst_target_treino = list()
lst_batch_ref_atms_treino = list()


lst_features_valid = list()
lst_distancias_valid = list()
lst_mol_sizes_valid = list()
lst_target_valid = list()
lst_batch_ref_atms_valid = list()

In [ ]:
print("Loading Training data ...")
inicio = time.time()
######################## Train Data #############################
for idx in tqdm(range(int(len(qm9data.train_dataset)))):
  val_target = qm9data.train_dataset[idx][aux_prop].tolist()[0]
  val_dist = Distance_Matrix(np.array(qm9data.train_dataset[idx]['_positions'].tolist()), 29)
  val_mol_size = len(qm9data.train_dataset[idx]['_atomic_numbers'].tolist())
  val_atm_numbers = qm9data.train_dataset[idx]['_atomic_numbers'].tolist()

  atomrefs = qm9data.train_dataset.atomrefs
  aux_ref_target_val = 0
  for atm_aux in qm9data.train_dataset[idx]['_atomic_numbers'].tolist():
    aux_ref_target_val = aux_ref_target_val + atomrefs[aux_prop][atm_aux].item()


  lst_features_treino.append(FeatureMatrix(val_atm_numbers, 29))
  lst_target_treino.append(val_target)
  lst_distancias_treino.append(val_dist)
  lst_batch_ref_atms_treino.append(aux_ref_target_val)
  lst_mol_sizes_treino.append(val_mol_size)

######################## Validation Data #############################
for idx in tqdm(range(int(len(qm9data.val_dataset)))):
  val_target = qm9data.val_dataset[idx][aux_prop].tolist()[0]
  val_dist = Distance_Matrix(np.array(qm9data.val_dataset[idx]['_positions'].tolist()), 29)
  val_mol_size = len(qm9data.val_dataset[idx]['_atomic_numbers'].tolist())
  val_atm_numbers = qm9data.val_dataset[idx]['_atomic_numbers'].tolist()

  atomrefs = qm9data.val_dataset.atomrefs
  aux_ref_target_val = 0
  for atm_aux in qm9data.val_dataset[idx]['_atomic_numbers'].tolist():
    aux_ref_target_val = aux_ref_target_val + atomrefs[aux_prop][atm_aux].item()


  lst_features_valid.append(FeatureMatrix(val_atm_numbers, 29))
  lst_target_valid.append(val_target)
  lst_distancias_valid.append(val_dist)
  lst_batch_ref_atms_valid.append(aux_ref_target_val)
  lst_mol_sizes_valid.append(val_mol_size)

fim = time.time()
print("Time(s) for data loading:", fim-inicio)

Loading Training data ...


100%|██████████| 10000/10000 [02:38<00:00, 62.96it/s]

Time(s) for data loading: 1897.2145600318909


# Target Variable transformation
* the schnetpack provides the data standarization process but, for the sake of clarity let's do it manually.
* $t_{m}$ = molecule target value without transformation
* $A_{m}$ = number of atoms in the molecule m
* $\widetilde{t}_{m} = \frac{t_{m} - ref_{m}}{A_{m}} $
* $\bar{\widetilde{t}} = \frac{1}{n} \cdot \sum_{m=1}^{n} \widetilde{t}_{m}$
* $\sigma_{\widetilde{t}}^{2} = \frac{1}{(n-1)} \sum_{m=1}^{n} (\widetilde{t}_{m} - \bar{\widetilde{t}})^{2}$
* $t_{m} = ref_{m} + A_{m} \cdot \bar{\widetilde{t}} + t'_{m} \cdot \sigma_{\widetilde{t}}$

In [ ]:
### Train Data
lst_target_treino = [(lst_target_treino[i] - lst_batch_ref_atms_treino[i])/lst_mol_sizes_treino[i] for i in range(len(lst_target_treino))]
mean_target_aux = np.mean(lst_target_treino)
std_target_aux = np.std(lst_target_treino)
# here it was used Standard deviation = 1
lst_target_treino = [lst_mol_sizes_treino[i]*(lst_target_treino[i]-mean_target_aux)/1 for i in range(len(lst_target_treino))]
print("Statistics for TRAIN data:    ", "Mean per Atom:", mean_target_aux, "        Std per Atom:", std_target_aux)


## Validation Data
lst_target_valid = [(lst_target_valid[i] - lst_batch_ref_atms_valid[i])/lst_mol_sizes_valid[i] for i in range(len(lst_target_valid))]
mean_target_aux_valid = np.mean(lst_target_valid)
std_target_aux_valid = np.std(lst_target_valid)
# here it was used Standard deviation = 1
lst_target_valid = [lst_mol_sizes_valid[i]*(lst_target_valid[i]-mean_target_aux_valid)/1 for i in range(len(lst_target_valid))]

print("Statistics for Validation data:    ", "Mean per Atom:", mean_target_aux_valid, "        Std per Atom:", std_target_aux_valid)

Statistics for TRAIN data:     Mean per Atom: -4.293860158116252         Std per Atom: 0.1887221180322883
Statistics for Validation data:     Mean per Atom: -4.2934503105520365         Std per Atom: 0.18872990358040848


# Model Training

In [ ]:
NeuralNetModel = AtomNeuralNet()

In [ ]:
TrainModel(NeuralNetModel, lst_features_treino,
          lst_target_treino, lst_distancias_treino,
          lst_mol_sizes_treino, 1,
          lst_features_valid, lst_target_valid,
          lst_distancias_valid, lst_mol_sizes_valid,
          n_epochs = 600, n_batch = 64)

Epoch: 1 MAE Train: 1.0164971441295045       MAE Validation: 0.932168175648337       Time(s): 89.19737029075623 
Epoch: 2 MAE Train: 0.41033921030447434       MAE Validation: 0.4768471640705876       Time(s): 85.74617290496826   Weights Updated :-)
Epoch: 3 MAE Train: 0.4613795219253995       MAE Validation: 0.5812620541738651       Time(s): 85.54584884643555 
Epoch: 4 MAE Train: 0.286695780338455       MAE Validation: 0.27187297704084196       Time(s): 85.6952896118164   Weights Updated :-)
Epoch: 5 MAE Train: 0.2721356396549489       MAE Validation: 0.2480964243338769       Time(s): 85.69433355331421 
Epoch: 6 MAE Train: 0.27977139173072585       MAE Validation: 0.2584078347223968       Time(s): 85.57968306541443 
Epoch: 7 MAE Train: 0.3589219668146611       MAE Validation: 0.3180087548763987       Time(s): 85.48248624801636 
Epoch: 8 MAE Train: 0.1913158912268611       MAE Validation: 0.18802557477938134       Time(s): 85.75999736785889   Weights Updated :-)
Epoch: 9 MAE Train: 0.17

# Inference
* Predict target value in test dataset

In [ ]:
lst_target_test = list()
lst_features_test = list()
lst_distancias_test = list()
lst_mol_sizes_test = list()

lst_mol_reference_test = list()

In [ ]:
for idx in tqdm(range(int(len(qm9data.test_dataset)))):

  val_target = qm9data.test_dataset[idx][aux_prop].tolist()[0]
  val_dist =  np.array([Distance_Matrix(np.array(qm9data.test_dataset[idx]['_positions'].tolist()), 29)])
  val_mol_size = len(qm9data.test_dataset[idx]['_atomic_numbers'].tolist())
  val_atm_numbers = qm9data.test_dataset[idx]['_atomic_numbers'].tolist()

  atomrefs = qm9data.test_dataset.atomrefs
  aux_ref_target_val = 0
  for atm_aux in qm9data.test_dataset[idx]['_atomic_numbers'].tolist():
    aux_ref_target_val = aux_ref_target_val + atomrefs[aux_prop][atm_aux].item()


  lst_features_test.append(np.array([FeatureMatrix(val_atm_numbers, 29)]))
  lst_distancias_test.append(val_dist)
  lst_mol_sizes_test.append(val_mol_size)
  lst_target_test.append(val_target)
  lst_mol_reference_test.append(aux_ref_target_val)

100%|██████████| 13885/13885 [03:37<00:00, 63.93it/s]


In [ ]:
pdct_test = Inference(NeuralNetModel, lst_features_test, lst_distancias_test, lst_mol_sizes_test)
pdct_test_inverse_transformation = np.array(lst_mol_reference_test) + np.array(lst_mol_sizes_test)*mean_target_aux + np.array(pdct_test)*1
print("Mean Absolute Error:", np.mean(abs(np.array(lst_target_test) - pdct_test_inverse_transformation)))

Mean Absolute Error: 0.015344434681756848
